In [ ]:
from model.BPRMF import *
from preprocess.Yelp import *
from evaluation.BPRMF_evaluation import *
pd.options.display.max_rows = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = './dataset/yelp2018'
dataset = Yelp(path)

# Data(num_nodes=144242, edge_index=[2, 2380730], edge_label_index=[2, 603378])
data = dataset.get()
num_users, num_books = dataset.getNumber()
config = {
    'k': 20,
    'lr':0.001,
    'epochs': 100,
    'num_layers': 2,
    'batch_size': 8192,
    'embedding_dim': 64,
    'num_users': num_users,
    'num_books': num_books,
    'tuning_type': None
}
model = BPRMF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    embedding_dim = config['embedding_dim']
).to(device)

In [ ]:
# Basic setting
num_edges = data.edge_index.size(1)
perm = torch.randperm(num_edges)
split = int(num_edges * 0.1)  # 10% of edges will be retained

# Define the data for forget and retain dataset
forget_data = Data()
retain_data = Data()

forget_data.num_nodes = data.num_nodes
retain_data.num_nodes = data.num_nodes
forget_data.edge_index = data.edge_index[:, perm[:split]]
retain_data.edge_index = data.edge_index[:, perm[split:]]
forget_data.edge_label_index = data.edge_label_index
retain_data.edge_label_index = data.edge_label_index

In [ ]:
config['epochs'] = 1000
retrain_model = BPRMF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    embedding_dim = config['embedding_dim']
).to(device)
retrain_model, recall, ndcg = BPRMF_eva(retrain_model, config, retain_data, device)

In [ ]:
BPRMF_forget_data_eva(retrain_model, None, forget_data, num_users, config['k'], config['batch_size'], device)

In [ ]:
# Define the model
teacher = BPRMF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    embedding_dim = config['embedding_dim']
).to(device)
student = BPRMF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    embedding_dim = config['embedding_dim']
).to(device)

# Load the model
teacher.load_state_dict(torch.load(f"BPRMF_Yelp2018_{config['epochs']}_Epochs_Top_{config['k']}.pt"))
student.load_state_dict(torch.load(f"BPRMF_Yelp2018_{config['epochs']}_Epochs_Top_{config['k']}.pt"))

In [ ]:
# Setting the basic hyperparameters
config['beta'] = 0.7
config['alpha'] = 0.3
config['epochs'] = 50
config['gamma'] = 1e-6 # contrastive loss
config['delta'] = 1e-3 # regularization loss
config['tuning_type'] = 'gpf'
config['weight_decay'] = 0.001
config['regularization'] = False
config['Contrastive_loss'] = False
student, prompt, epoch_tracks, test_topks = prompt_BPRMF_unlearning_eva(teacher, student, config, retain_data, forget_data, device)

In [ ]:
BPRMF_forget_data_eva(student, prompt, forget_data, num_users, config['k'], config['batch_size'], device)

In [ ]:
# Setting the basic hyperparameters
config['contrastive_loss'] = False
config['regularization'] = True

student, prompt, epoch_tracks, test_topks = prompt_BPRMF_unlearning_eva(teacher, student, config, retain_data, forget_data, device)

In [ ]:
BPRMF_forget_data_eva(student, prompt, forget_data, num_users, config['k'], config['batch_size'], device)

In [ ]:
# Setting the basic hyperparameters
config['contrastive_loss'] = True
config['regularization'] = False

student, prompt, epoch_tracks, test_topks = prompt_BPRMF_unlearning_eva(teacher, student, config, retain_data, forget_data, device)

In [ ]:
BPRMF_forget_data_eva(student, prompt, forget_data, num_users, config['k'], config['batch_size'], device)

In [ ]:
# Setting the basic hyperparameters
config['contrastive_loss'] = True
config['regularization'] = True

student, prompt, epoch_tracks, test_topks = prompt_BPRMF_unlearning_eva(teacher, student, config, retain_data, forget_data, device)

In [ ]:
BPRMF_forget_data_eva(student, prompt, forget_data, num_users, config['k'], config['batch_size'], device)

In [ ]:
config['tuning_type'] = 'gpf-plus'
config["number_p"] = 10 # The number of prompts
# Setting the basic hyperparameters
config['contrastive_loss'] = True
config['regularization'] = True

student, prompt, epoch_tracks, test_topks = prompt_BPRMF_unlearning_eva(teacher, student, config, retain_data, forget_data, device)

In [ ]:
BPRMF_forget_data_eva(student, prompt, forget_data, num_users, config['k'], config['batch_size'], device)